In [1]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import networkx as nx
from IPython.display import Image
from pprint import pprint

In [2]:
import osmnx as ox
ox.config(log_console=True, use_cache=True)
ox.__version__

'1.1.2'

In [ ]:
G = ox.graph_from_place("Kamppi, Helsinki, Finland", network_type="drive")
fig, ax = ox.plot_graph(G)

C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [ ]:
# project graph and calculate its convex hull area
G_proj = ox.project_graph(G)
nodes_proj = ox.graph_to_gdfs(G_proj)
graph_area_m = nodes_proj.unary_union.convex_hull.area

In [ ]:
# calculate and print basic stats
stats = ox.basic_stats(G_proj, area=graph_area_m, clean_intersects=True,
                       circuity_dist="euclidean")
pprint(stats)

In [ ]:
# calculate and print extended stats
more_stats = ox.extended_stats(G, ecc=True, bc=True, cc=True)
for key in sorted(more_stats.keys()):
    print(key)

### visualizing street centrality

In [ ]:
edge_centrality = nx.closeness_centrality(nx.line_graph(G))

In [ ]:
# make a list of graph edge centrality values
ev = [edge_centrality[edge + (0,)] for edge in G.edges()]

# create a color scale converted to list of colors for graph edges
norm = colors.Normalize(vmin=min(ev)*0.8, vmax=max(ev))
cmap = cm.ScalarMappable(norm=norm, cmap=cm.inferno)
ec = [cmap.to_rgba(cl) for cl in ev]

# color the edges in the original graph by closeness centrality in the line graph
fig, ax = ox.plot_graph(G, bgcolor="black", axis_off=True, node_size=0,
                       edge_colors=ec, edge_linewidth=2, edge_alpha=1)